# TensorFlow Basics

Content From

https://www.tensorflow.org/get_started/get_started

## Veiwing this notebook correctly and making tensorflow snippets work

The steps below would help executing the contents of this notebook
- Create a new conda env. eg. "tensorflow"
- Follow the steps from this link
https://www.tensorflow.org/install/install_windows#installing_with_anaconda 
Perferably install the non-gpu version ( cuda doesn't like me )
- Install Jupyter Notebook in the conda env "tensorflow"

## Verify TensorFlow works in this notebook

In [1]:
import tensorflow as tf
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

b'Hello, TensorFlow!'


## What is a tensor ?

A Tensor can be thought of like a n-d matrix

~~~
  3   // Zero Dimensional Matrix- A scaler - a rank 0 tensor
 [3]  // One Dimensional Matrix- A Vector - a rank 1 tensor
[[3]] // Two Dimensional Matrix- A Vector - a rank 2 tensor
~~~

## TensorFlow objects

TensorFlow operations are performed into a graph of nodes.  Each node takes zero or more tensors as inputs and produces a tensor as an output. Nodes can of following types
- Constants
- PlaceHolder
- Variable

## Constants

One type of node is a constant. Like all TensorFlow constants, it takes no inputs, and it outputs a value it stores internally.

In [2]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)


Tensor("Const_1:0", shape=(), dtype=float32) Tensor("Const_2:0", shape=(), dtype=float32)


Printing the nodes does not output the values 3.0 and 4.0 as you might expect. Instead, they are nodes that, when evaluated, would produce 3.0 and 4.0, respectively. To actually evaluate the nodes, we must run the computational graph within a session. A session encapsulates the control and state of the TensorFlow runtime.

In [3]:
sess = tf.Session()
print(sess.run([node1, node2]))


[3.0, 4.0]


In the snippet below we assign the outcome of tf.add() to another node.

We evaluate the new node by sess.run() 

In [5]:
from __future__ import print_function
node3 = tf.add(node1, node2)
print( "node3:", node3 ) # no evalution done at this step
print("sess.run(node3):", sess.run(node3)) # value of node 3 evaluated now

node3: Tensor("Add_1:0", shape=(), dtype=float32)
sess.run(node3): 7.0


## PlaceHolder

A graph can be parameterized to accept external inputs, known as placeholders. A placeholder is a promise to provide a value later.



In [7]:
# create two placeholder variables
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

# provie their value via dict and evalute the adder_node
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))


7.5
[3. 7.]


## Variable

PlaceHolders enable us to create a model that can take arbitrary inputs, such as the one above. To make the model trainable, we need to be able to modify the graph to get new outputs with the same input. Variables allow us to add trainable parameters to a graph.

In [8]:
# create two varaibles
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)

# a placeholder
x = tf.placeholder(tf.float32)

# Model is a combination of varaible and placeholder
linear_model = W*x + b

# Constants are initialized when you call tf.constant, and their value can never change. 
# By contrast, variables are not initialized when you call tf.Variable.  
# To initialize all the variables in a TensorFlow program, you must explicitly call a special operation as follows:
init = tf.global_variables_initializer()
sess.run(init)

# Since x is a placeholder, we can evaluate linear_model for several values of x simultaneously as follows
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))


[0.         0.3        0.6        0.90000004]


## Loss Function 

We've created a model, but we don't know how good it is yet. To evaluate the model on training data, we need a y placeholder to provide the desired values, and we need to write a loss function.

A loss function measures how far apart the current model is from the provided data. We'll use a standard loss model for linear regression, which sums the squares of the deltas between the current model and the provided data. linear_model - y creates a vector where each element is the corresponding example's error delta. We call tf.square to square that error. Then, we sum all the squared errors to create a single scalar that abstracts the error of all examples using tf.reduce_sum

In [9]:
# Create a place holder variable
# The input to this variable represents how our ideal model should be
# In the above snippet we evaluated linear_model for several values of x simultaneously as follows
# [ 0, 0.3, 0.6, 0.9 ]
# The variable y computes the loss function wrt to the above linear model
y = tf.placeholder(tf.float32)

# standard
squared_deltas = tf.square(linear_model - y)

loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))


23.66


# Bringing the Loss down to zero

We could improve this manually by reassigning the values of W and b to the perfect values of -1 and 1. A variable is initialized to the value provided to tf.Variable but can be changed using operations like tf.assign. For example, W=-1 and b=1 are the optimal parameters for our model. We can change W and b accordingly:



In [10]:
# although W and b were assigned some values initially
# we can fix them by providing specific values
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])

# fix the session to make sure fiwW and fixB get correct values
sess.run([fixW, fixb])

# compute the loss function again
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))


0.0


## tf.train API

In the snippet above we guessed W and B to -1 and 1 to make the loss function return 0, can this be done automatically is the whole point of TensorFlow.

TensorFlow provides optimizers that slowly change each variable ( W and b ) in order to minimize the loss function. The simplest optimizer is gradient descent. It modifies each variable according to the magnitude of the derivative of loss with respect to that variable. 

In nutshell tensorflow figures out for us the optimal values for W and b

In [11]:

# Model parameters
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W*x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
  sess.run(train, {x: x_train, y: y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))


W: [-0.9999969] b: [0.9999908] loss: 5.6999738e-11
